### Notebook for presentation at Recognising Refugees: Comparative and Transnational Insights into Asylum under Pressure (Dublin 2025)

This notebook provides an example of how to extract information from Canadian refugee law documents at scale using large language models, including both commerical models (OpenAI) and open source models (qwen via ollama).

In [ ]:
# pip install pandas tqdm datasets openai ollama python-dotenv

# general imports
from time import sleep

# setup pandas
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# setup access to huggingface data
from datasets import load_dataset

In [ ]:
# get the data
df = load_dataset("refugee-law-lab/canadian-legal-data", data_dir = "RAD", split = "train").to_pandas()

# filter for language == 'en'
df = df[df['language'] == 'en']

# in df.unofficial_text remove footnotes
df['unofficial_text'] = df['unofficial_text'].str.replace(r'\n(?:\d+ .+?)(?=\n)', '', regex=True)

# remove header if REASONS FOR DECISION is in the text
print("Removing header")
def remove_header(text):
    before, sep, after = text.partition('REASONS FOR DECISION')
    if sep:  # found the string
        return sep + after
    else:
        return text  # leave untouched if not found
df['unofficial_text'] = df['unofficial_text'].apply(remove_header)

# create sample to get outcomes for all cases
df_sample = df.copy()
df_sample = df_sample.sample(n=5000, random_state=888).reset_index(drop=True)

df.head(5)


In [ ]:
# keyword search for cases realted to sexual orientation or gendier identity/expression
keywords = [
    'lgbt',
    'queer',
    'sogie',
    'sexual orienation',
    'gay',
    'homosexual',
    'lesbian',
    'bisexual',
    'bi-sexual',
    'pansexual',
    'pan-sexual',
    'same-sex',
    'same sex',
    'transgender',
    'transexual',
    'transman',
    'transwoman',
    'mtf',
    'ftm',
    'non-binary',
    'gender identity',
    'gender expression',
    'genderqueer',
    'gender queer',
    'gender fluid',
    'gender-fluid',
    'gender non-conforming',
    'gender nonconforming',
    'gender expansive',
    'two spirit'
    ]

def keyword_search(text):
    for kw in keywords:
        if kw in text.lower():
            return True
    return False
print("Searching for keywords")
df['keyword_search'] = df['unofficial_text'].progress_apply(keyword_search)

print ("Length of en df before keyword search: ", len(df))
df = df[df['keyword_search'] == True]
df = df.drop(columns=['keyword_search'])
df = df.reset_index(drop=True)

print ("Length of en df after keyword search: ", len(df))


In [ ]:
# setup ollama
# NOTE: Need to install ollama on system: https://ollama.com/download
# Set up ollama server (ollama serve), and download the models you want to use (ollama pull <model>)
# If you prefer not running ollama server, you can use transformers
from ollama import Client as OllamaClient

# setup openai
# NOTE: Need to create .env file with OPENAI_API_KEY=xxxxxxxx
# Get your OpenAI API key from https://platform.openai.com
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# fuctions to call llms

def get_ollama_completion(prompt,
                 model = 'qwen2.5:72b',
                 temperature = 0,
                 num_predict = 1,
                 host = 'http://ts-ollama:11434',  #For local use, change to 'http://localhost:11434'
                 attempts = 3
                 ):  

    for x in range(attempts):   
        try:
            client = OllamaClient(host=host)
            response = client.generate(
                model=model,
                prompt=prompt,
                options={"temperature": temperature, "num_predict": num_predict}
            )
            sleep(.1) # slow down requests to avoid problems with ollama server
            return response["response"]
    
        except:
            print("Error in connection. Trying again after 10 seconds")
            sleep(10)
            if x == attempts - 1:
                print("Too many errors. Returning empty string.")
                return ""


def get_openai_completion(user_message,
        system_message="You are a helpful assistant to a Canadian law student",
        model = "gpt-4o-mini",
        temperature = 0,
        num_predict = 1):
    client = OpenAI()
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        max_completion_tokens = num_predict,
    )
    return completion.choices[0].message.content

print("Ollama:", get_ollama_completion("If you hear me, say 'working' and nothing elese"))
print("OpenAI:", get_openai_completion("If you hear me, say 'working' and nothing elese"))

In [ ]:
# set up prompts

def get_prompt(docs, question = "Summarize the document.\n\n"):
    prompt = f"""CONTEXT: You are a legal assistant. You are provided a document and you are asked
a question about that document. You only answer the question with no explanation

DOCUMENT:
{docs}

QUESTION:
{question}

ANSWER:
"""
    return prompt

In [ ]:
outcome_question = """True or false: In the document provided, the appeal is granted. Only answer with True or False."""

# apply to df to get ollama response (use smaller model for speed):
df_sample['ollama_outcome'] = df_sample.progress_apply(
    lambda x: get_ollama_completion(get_prompt(x['unofficial_text'], outcome_question), model='qwen2.5:32b'),
    axis=1)

# export to json
df_sample.to_json("RAD_sample_outcome.json", orient="records", indent = 4)

In [ ]:
# test for accuracy against openai gpt-4o-mini

df_sample = pd.read_json("RAD_sample_outcome.json", orient="records")

df_sample = df_sample.sample(100)

outcome_question = """True or false: In the document provided, the appeal is granted. Only answer with True or False."""

# apply to df to get openai gpt-4o-mini response:
df_sample['openai_outcome'] = df_sample.progress_apply(
    lambda x: get_openai_completion(get_prompt(x['unofficial_text'], outcome_question), model = 'gpt-4o-mini'), 
    axis=1)

# len where ollama_outcome != openai_outcome
print("Differences etween openai and ollama", len(df_sample[df_sample['ollama_outcome'] != df_sample['openai_outcome']]))

# export small sample to json
df_sample.to_json("RAD_small_sample_outcome_comparison.json", orient="records", indent = 4)

In [ ]:

type_question = """Multiple choice (return only the number): You are assisting a law professor in categorizing
refugee law cases involving persecution based on sexual orientation or gender identity/expression. The professor
is interested in the principal claimant's *allegations*, not whether these allegations are true or credible.

Your task is to identify the category that best describes the particular social group the principal 
claimant *alleges* to belong to, based on the information in the document.

Select the most appropriate category (if applicable):

1. Gay man / homosexual (excluding men who also identify as bisexual or pansexual)
2. Lesbian woman (excluding women who also identify as bisexual or pansexual)
3. Bisexual / pansexual man (including men who identiy as *both* gay and bisexual)
4. Bisexual / pansexual woman  (including women who identify as *both* lesbian and bisexual)
5. Transgender / non-binary person  
6. Other: A principal claim involving sexual orientation or gender identity/expression that does not fit categories 1-5 (including, e.g. family members or friends of LGBTQ+ individuals)
7. Other: A principal claim that does *not* involve sexual orientation or gender identity/expression

Be sure to return the number corresponding to your choice in your final answer. Do not return any other text or explanation."""

# apply to df to get ollama response (use bigger model for accuracy)
df['ollama_type'] = df.progress_apply(
    lambda x: get_ollama_completion(get_prompt(x['unofficial_text'], type_question), model = 'qwen2.5:72b'), 
    axis=1)

# apply to df to get openai gpt-4o-mini response:
df['openai_type'] = df.progress_apply(
    lambda x: get_openai_completion(get_prompt(x['unofficial_text'], type_question), model = 'gpt-4o-mini'), 
    axis=1)



In [ ]:
outcome_question = """True or false: In the document provided, the appeal is granted. Only answer with True or False."""

# apply to df to get ollama response (use smaller model for speed):
df['ollama_outcome'] = df.progress_apply(
    lambda x: get_ollama_completion(get_prompt(x['unofficial_text'], outcome_question), model = 'qwen2.5:32b'), 
    axis=1)


In [ ]:
# export to json
df.to_json("RAD_SOGIE_type_outcome.json", orient="records", indent = 4)

### Analyze data

In [1]:
# import data

import pandas as pd

df_small_sample = pd.read_json("RAD_small_sample_outcome_comparison.json")
df_sample = pd.read_json("RAD_sample_outcome.json")
df = pd.read_json("RAD_SOGIE_type_outcome.json")

In [2]:
# compare ollama and openai outcomes in df_sample
differences = df_small_sample[df_small_sample['ollama_outcome'] != df_small_sample['openai_outcome']]
print("Differences between openai and ollama:", len(differences), "out of", len(df_small_sample))

Differences between openai and ollama: 2 out of 100


So, we see that the small qwen model agrees with the larger openai model about the
outcome of RAD cases 98% of the time, so we'll rely on the small qwen model

In [8]:
# Stats on ollama outcomes in df_sample
print("Out of 5000 RAD cases in radom sample,", df_sample['ollama_outcome'].value_counts()["True"], "or", round(df_sample['ollama_outcome'].value_counts()["True"]/len(df_sample)*100, 1), "% of cases, are granted")

Out of 5000 RAD cases in radom sample, 1369 or 27.4 % of cases, are granted


So, the sucess rate on appeal in the random sample of 5,000 RAD cases (across all claim types) is: 27.4%

In [10]:
# number of rows were df.ollama == df.openai
print("Types differnces qwen v openai:", len(df[df['ollama_type'] != df['openai_type']]), "out of", len(df), "or", round(len(df[df['ollama_type'] != df['openai_type']])/len(df)*100, 1), "% of cases")

Types differnces qwen v openai: 309 out of 1717 or 18.0 % of cases


In contrast to outcomes, qwen and openai regularly disagree about what claim type classification best applies to a case (18% of the time). We could try several differnt approaches, including using larger models, using multiple models with voting, using reasoning models, etc. For convenience, we'll just remove the cases where openai and qwen disagree for now

In [11]:
# remove rows where there are differences between ollama and openai
df = df[df['ollama_type'] == df['openai_type']]
df = df.reset_index(drop=True)
print("Number of rows in df after removing differences:", len(df))

Number of rows in df after removing differences: 1408


Beause we're only interested in SOGIE cases, we will also remove cases where the classification is other

In [12]:
# remove rows where type is 6 or 7 (i.e. not claim where claimant is LGBTQ+)
df = df[(df['ollama_type'] != 6) & (df['ollama_type'] != 7)]
df = df.reset_index(drop=True)
print("Number of rows in df after removing type 6 and 7:", len(df))

Number of rows in df after removing type 6 and 7: 1188


So, that leaves us with 1,188 SOGIE cases. Let's look at outcomes in those cases:

In [19]:
ollama_types = {1:"Gay man", 2: "Lesbian", 3: "Bisexual man", 4: "Bisexual woman", 5: "Transgeder"}

# iterate through dict
for ollama_type in ollama_types:
    num_claims = len(df[df['ollama_type'] == ollama_type])
    percent_claims = len(df[df['ollama_type'] == ollama_type])/len(df)*100
    success_rate = len(df[(df['ollama_type'] == ollama_type) & (df['ollama_outcome'] == "True")])/num_claims*100
    print(f"{ollama_types[ollama_type]} claims: {num_claims} ({percent_claims:.1f}%), with success rate of {success_rate:.1f}%")

# same but for all types
num_claims = len(df)
percent_claims = len(df)/len(df)*100
success_rate = len(df[df['ollama_outcome'] == "True"])/num_claims*100
print()
print(f"All SOGIE claims: {num_claims}, with success rate of {success_rate:.1f}%")
print()
print("All claims in random sample for all types: 5,000 with success rate of", round(df_sample['ollama_outcome'].value_counts()["True"]/len(df_sample)*100, 1), "%")
    
    

Gay man claims: 521 (43.9%), with success rate of 35.9%
Lesbian claims: 192 (16.2%), with success rate of 34.9%
Bisexual man claims: 293 (24.7%), with success rate of 37.5%
Bisexual woman claims: 177 (14.9%), with success rate of 42.9%
Transgeder claims: 5 (0.4%), with success rate of 60.0%

All SOGIE claims: 1188, with success rate of 37.3%

All claims in random sample for all types: 5,000 with success rate of 27.4 %


In the end, we see that SOGIE claims are much more likely to result in successful appeals than other types of claims, suggesting that the refugee determination systems struggles to get these claims right at first instance -- and that different subtypes of SOGIE claims also have varying success rates on appeal.